In [4]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns

In [5]:
df = pd.read_csv('total_data.csv')

### xdata로 상관계수가 높은 column을 넣어서 Ridge
- elasticnet으로 상관계수가 높은 feature를 넣어 모델생성

In [6]:
# 필요 패키지 로드
#from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error

In [7]:
# y값인 q1-q5가 결측인 2020년 데이터 제거
a = df[0:-82]
a

,jur_stn,year,child,house,report_sx_total,vio_cnt,mur_rob_cnt,car_cnt,popu_합계,for_합계,...,vic_w,vic_unknown,case_car_man,case_car_car,case_car_only,q1,q2,q3,q4,q5
0,서울중부경찰서,2017(상반기),39.0,97.0,22574.0,1324.0,0.0,1412.0,63948.0,5132.5,...,0.241636,0.022305,0.274074,0.703704,0.022222,75.2,76.5,69.9,65.9,72.6
1,서울종로경찰서,2017(상반기),46.0,98.0,19674.0,1102.0,0.0,867.0,78638.5,5489.5,...,0.262745,0.039216,0.282353,0.682353,0.035294,81.6,82.4,69.1,62.7,72.2
2,서울남대문경찰서,2017(상반기),39.0,97.0,14499.0,949.0,0.0,736.0,63948.0,5132.5,...,0.203922,0.043137,0.247059,0.713725,0.039216,77.3,79.3,68.8,61.0,71.7
3,서울서대문경찰서,2017(상반기),39.0,98.0,38769.0,1924.0,0.0,1788.0,321345.0,13152.0,...,0.243697,0.044118,0.266807,0.691176,0.042017,78.0,80.0,67.0,66.3,71.4
4,서울혜화경찰서,2017(상반기),46.0,98.0,18857.0,1127.0,0.0,899.0,78638.5,5489.5,...,0.217054,0.046512,0.262548,0.694981,0.042471,78.6,80.6,70.7,65.2,75.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,창원서부경찰서,2019(하반기),44.0,87.0,22771.0,1460.0,2.0,1964.0,256001.0,5100.0,...,0.286982,0.047337,0.218935,0.733728,0.047337,79.1,79.5,68.5,71.2,76.0
242,마산중부경찰서,2019(하반기),37.0,88.0,21264.0,1265.0,5.0,1584.0,177579.0,3145.0,...,0.280000,0.092308,0.258462,0.649231,0.092308,80.8,80.0,70.8,73.2,76.5
243,마산동부경찰서,2019(하반기),44.0,86.0,26014.0,1715.0,8.0,1910.0,192446.0,2871.0,...,0.345679,0.032922,0.345679,0.621399,0.032922,75.0,76.5,69.4,68.9,73.1
244,진주경찰서,2019(하반기),64.0,92.0,38557.0,2552.0,0.0,3570.0,354166.0,7152.0,...,0.357320,0.057072,0.357320,0.588089,0.054591,78.7,79.6,70.3,74.6,75.6


In [8]:
# 경찰서와 연도 데이터 제거
a.drop(columns = ['jur_stn', 'year'], inplace = True)

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [9]:
a_ = a.drop(columns = ['q1', 'q2', 'q3', 'q4', 'q5'])

In [10]:
a_.columns

Index(['child', 'house', 'report_sx_total', 'vio_cnt', 'mur_rob_cnt',
       'car_cnt', 'popu_합계', 'for_합계', 'popu_u20', 'popu_o60', 'for_u20',
       'for_2030', 'single_합계', 'single_u20', 'single_2030', 'single_4050',
       'single_o60', 'single_f_합계', 'single_f_u20', 'single_f_2030',
       'single_f_4050', 'single_f_o60', 'popu_2050', 'for_o40', '총범죄발생대비검거수',
       '절도폭력범죄발생대비검거수', '강도살인범죄발생대비검거수', '교통사고범죄발생대비검거수', '기초질서범죄발생대비검거수',
       '인구수대비검거수', '외국인인구수대비검거수', '인구수대비경찰수', 'before_q1', 'before_q2',
       'before_q3', 'before_q4', 'before_q5', 'light_r', 'cctv_r', 'adult_r',
       'bell_r', 'ofn_10', 'ofn_20', 'ofn_340', 'ofn_5670', 'ofn_80ov',
       'ofn_m', 'ofn_w', 'ofn_unknown', 'vic_10', 'vic_20', 'vic_340',
       'vic_5670', 'vic_80ov', 'vic_m', 'vic_w', 'vic_unknown', 'case_car_man',
       'case_car_car', 'case_car_only'],
      dtype='object')

In [11]:
a_1 = a_

In [12]:
a_2 = a_

In [13]:
a_3 = a_

In [14]:
a_4 = a_

In [15]:
a_5 = a_

In [16]:
# StandardScaling
scaler1 = StandardScaler()
scaler1.fit(a_1) # scaler에 xdata 학습
a_s1 = scaler1.transform(a_1)
a_s1

array([[-1.16204451, -1.00794068, -0.94618196, ...,  0.39434438,
        -0.11566933, -0.96802532],
       [-0.81221223, -0.8173539 , -1.12728457, ...,  0.56656445,
        -0.52850194,  0.00726295],
       [-1.16204451, -1.00794068, -1.45045906, ..., -0.16763689,
         0.07810924,  0.29984943],
       ...,
       [-0.91216431, -3.10439533, -0.73135679, ...,  1.88389689,
        -1.70708755, -0.16973381],
       [ 0.08735649, -1.96087461,  0.05194325, ...,  2.1260591 ,
        -2.35115781,  1.44696268],
       [-1.26199659, -2.53263497, -1.0869424 , ..., -1.10883527,
         0.83430992,  0.7576378 ]])

In [17]:
# StandardScaling
scaler2 = StandardScaler()
scaler2.fit(a_2) # scaler에 xdata 학습
a_s2 = scaler2.transform(a_2)
a_s2

array([[-1.16204451, -1.00794068, -0.94618196, ...,  0.39434438,
        -0.11566933, -0.96802532],
       [-0.81221223, -0.8173539 , -1.12728457, ...,  0.56656445,
        -0.52850194,  0.00726295],
       [-1.16204451, -1.00794068, -1.45045906, ..., -0.16763689,
         0.07810924,  0.29984943],
       ...,
       [-0.91216431, -3.10439533, -0.73135679, ...,  1.88389689,
        -1.70708755, -0.16973381],
       [ 0.08735649, -1.96087461,  0.05194325, ...,  2.1260591 ,
        -2.35115781,  1.44696268],
       [-1.26199659, -2.53263497, -1.0869424 , ..., -1.10883527,
         0.83430992,  0.7576378 ]])

In [18]:
# StandardScaling
scaler3 = StandardScaler()
scaler3.fit(a_3) # scaler에 xdata 학습
a_s3 = scaler3.transform(a_3)
a_s3

array([[-1.16204451, -1.00794068, -0.94618196, ...,  0.39434438,
        -0.11566933, -0.96802532],
       [-0.81221223, -0.8173539 , -1.12728457, ...,  0.56656445,
        -0.52850194,  0.00726295],
       [-1.16204451, -1.00794068, -1.45045906, ..., -0.16763689,
         0.07810924,  0.29984943],
       ...,
       [-0.91216431, -3.10439533, -0.73135679, ...,  1.88389689,
        -1.70708755, -0.16973381],
       [ 0.08735649, -1.96087461,  0.05194325, ...,  2.1260591 ,
        -2.35115781,  1.44696268],
       [-1.26199659, -2.53263497, -1.0869424 , ..., -1.10883527,
         0.83430992,  0.7576378 ]])

In [19]:
# StandardScaling
scaler4 = StandardScaler()
scaler4.fit(a_4) # scaler에 xdata 학습
a_s4 = scaler4.transform(a_4)
a_s4

array([[-1.16204451, -1.00794068, -0.94618196, ...,  0.39434438,
        -0.11566933, -0.96802532],
       [-0.81221223, -0.8173539 , -1.12728457, ...,  0.56656445,
        -0.52850194,  0.00726295],
       [-1.16204451, -1.00794068, -1.45045906, ..., -0.16763689,
         0.07810924,  0.29984943],
       ...,
       [-0.91216431, -3.10439533, -0.73135679, ...,  1.88389689,
        -1.70708755, -0.16973381],
       [ 0.08735649, -1.96087461,  0.05194325, ...,  2.1260591 ,
        -2.35115781,  1.44696268],
       [-1.26199659, -2.53263497, -1.0869424 , ..., -1.10883527,
         0.83430992,  0.7576378 ]])

In [20]:
# StandardScaling
scaler5 = StandardScaler()
scaler5.fit(a_5) # scaler에 xdata 학습
a_s5 = scaler5.transform(a_5)
a_s5

array([[-1.16204451, -1.00794068, -0.94618196, ...,  0.39434438,
        -0.11566933, -0.96802532],
       [-0.81221223, -0.8173539 , -1.12728457, ...,  0.56656445,
        -0.52850194,  0.00726295],
       [-1.16204451, -1.00794068, -1.45045906, ..., -0.16763689,
         0.07810924,  0.29984943],
       ...,
       [-0.91216431, -3.10439533, -0.73135679, ...,  1.88389689,
        -1.70708755, -0.16973381],
       [ 0.08735649, -1.96087461,  0.05194325, ...,  2.1260591 ,
        -2.35115781,  1.44696268],
       [-1.26199659, -2.53263497, -1.0869424 , ..., -1.10883527,
         0.83430992,  0.7576378 ]])

In [21]:
# x데이터 설정 2017, 2018 데이터를 학습용, 2019년 데이터를 검증용 데이터셋으로 설정

xtrain1 = a_s1[:-82]
xtest1 = a_s1[-82:]
xtrain2 = a_s2[:-82]
xtest2 = a_s2[-82:]
xtrain3 = a_s3[:-82]
xtest3 = a_s3[-82:]
xtrain4 = a_s4[:-82]
xtest4 = a_s4[-82:]
xtrain5 = a_s5[:-82]
xtest5 = a_s5[-82:]

In [22]:
# y데이터 설정 2017, 2018 데이터를 학습용, 2019년 데이터를 검증용 데이터셋으로 설정

train = a[:-82]
test = a[-82:]

ytrain1 = train['q1']
ytrain2 = train['q2']
ytrain3 = train['q3']
ytrain4 = train['q4']
ytrain5 = train['q5']

ytest1 = test['q1']
ytest2 = test['q2']
ytest3 = test['q3']
ytest4 = test['q4']
ytest5 = test['q5']

In [23]:
a_1.columns

Index(['child', 'house', 'report_sx_total', 'vio_cnt', 'mur_rob_cnt',
       'car_cnt', 'popu_합계', 'for_합계', 'popu_u20', 'popu_o60', 'for_u20',
       'for_2030', 'single_합계', 'single_u20', 'single_2030', 'single_4050',
       'single_o60', 'single_f_합계', 'single_f_u20', 'single_f_2030',
       'single_f_4050', 'single_f_o60', 'popu_2050', 'for_o40', '총범죄발생대비검거수',
       '절도폭력범죄발생대비검거수', '강도살인범죄발생대비검거수', '교통사고범죄발생대비검거수', '기초질서범죄발생대비검거수',
       '인구수대비검거수', '외국인인구수대비검거수', '인구수대비경찰수', 'before_q1', 'before_q2',
       'before_q3', 'before_q4', 'before_q5', 'light_r', 'cctv_r', 'adult_r',
       'bell_r', 'ofn_10', 'ofn_20', 'ofn_340', 'ofn_5670', 'ofn_80ov',
       'ofn_m', 'ofn_w', 'ofn_unknown', 'vic_10', 'vic_20', 'vic_340',
       'vic_5670', 'vic_80ov', 'vic_m', 'vic_w', 'vic_unknown', 'case_car_man',
       'case_car_car', 'case_car_only'],
      dtype='object')

In [24]:
# 그리드 서치 패키지
from sklearn.model_selection import GridSearchCV

In [25]:
# 최적 성능을 내는 lasso의 alpha값을 얻기 위해 param_grid 생성

param_grid = {'alpha' : np.linspace(0.001, 10.0, 10000)}

#그리드 서치 설정
#grid_search = GridSearchCV(Lasso(), param_grid = param_grid, cv = 10, n_jobs = -1, scoring ='r2')
grid_search = GridSearchCV(Lasso(), param_grid = param_grid, cv = 10, n_jobs = -1, scoring ='neg_mean_absolute_error')

#### q1 절도폭력 

In [26]:
# 그리드 서치 후 최고 성능의 모델을 ela1에 저장

grid_search.fit(xtrain1, ytrain1)
lasso1 = grid_search.best_estimator_

In [27]:
# MAE 출력

y_pred1 = lasso1.predict(xtest1)
mean_absolute_error(ytest1, y_pred1)

2.1871350084392516

In [46]:
# 결과

print('alpha =', lasso1.alpha)
print(lasso1.coef_) # Ridge 회귀분석으로 나온 weghit값
print('feature 개수: ', len(a_1.columns[lasso1.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_1.columns[lasso1.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_1.columns[lasso1.coef_.argmin()])

alpha = 0.021
[ 0.          0.42511649 -0.         -1.65910729  0.16948203  0.
  0.         -0.          0.85163212  0.99353712  1.47668504 -0.60711634
 -0.          0.3242491   0.         -1.28507164 -0.13031473  0.
  0.          1.09295358  0.         -0.          0.         -0.46686054
 -0.37708668 -0.10055692  0.01279539  0.04867457  0.19010976 -0.71763197
  1.41143962  1.18624975 -0.         -0.         -0.20921486  0.05442657
  0.30350117  0.17642624 -0.21026377 -0.21759718  0.18835426 -0.32880902
  0.          0.32464304 -0.         -0.02737931 -0.7174347   0.
  0.18052958 -0.18436065  0.          0.          0.         -0.3636099
 -0.          0.37375259 -0.          0.         -0.00529149  0.1508471 ]
feature 개수:  37
가장 강한 양의 상관관계:  for_u20 
가장 강한 음의 상관관계:  vio_cnt


In [30]:
print('feature: ', a_1.columns[lasso1.coef_ != 0])

feature:  Index(['house', 'vio_cnt', 'mur_rob_cnt', 'popu_u20', 'popu_o60', 'for_u20',
       'for_2030', 'single_u20', 'single_4050', 'single_o60', 'single_f_2030',
       'for_o40', '총범죄발생대비검거수', '절도폭력범죄발생대비검거수', '강도살인범죄발생대비검거수',
       '교통사고범죄발생대비검거수', '기초질서범죄발생대비검거수', '인구수대비검거수', '외국인인구수대비검거수', '인구수대비경찰수',
       'before_q3', 'before_q4', 'before_q5', 'light_r', 'cctv_r', 'adult_r',
       'bell_r', 'ofn_10', 'ofn_340', 'ofn_80ov', 'ofn_m', 'ofn_unknown',
       'vic_10', 'vic_80ov', 'vic_w', 'case_car_car', 'case_car_only'],
      dtype='object')


#### q2 강도살인

In [31]:
# 그리드 서치 후 최고 성능의 모델을 ela2에 저장

grid_search.fit(xtrain2, ytrain2)
lasso2 = grid_search.best_estimator_

In [32]:
# MAE 출력

y_pred2 = lasso2.predict(xtest2)
mean_absolute_error(ytest2, y_pred2)

2.0471413661020796

In [44]:
# 결과

print('alpha =', lasso2.alpha)
print(lasso2.coef_) # Ridge 회귀분석으로 나온 weghit값
print('feature 개수: ', len(a_2.columns[lasso2.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_2.columns[lasso2.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_2.columns[lasso2.coef_.argmin()])

alpha = 0.021
[ 0.          0.42915689 -0.         -2.01752393  0.          0.74594998
  0.         -0.          0.59198989  1.92049972  1.78388758 -1.04086226
  0.          0.37471984  0.08018347 -1.23172997 -0.59120425  0.
 -0.          0.91909285  0.         -0.          0.         -0.52745292
 -0.         -0.         -0.          0.06729044  0.19837992 -0.92741628
  0.70851213  1.36886505 -0.1356403   0.         -0.53753043  0.28916076
  0.45684253  0.10212748 -0.1538169  -0.18862435  0.01418414 -0.61367072
 -0.          0.3030273  -0.         -0.         -0.89692431  0.
  0.15673515  0.         -0.05722712  0.          0.         -0.5920882
 -0.          0.0435111  -0.          0.         -0.46052082  0.28183134]
feature 개수:  35
가장 강한 양의 상관관계:  popu_o60 
가장 강한 음의 상관관계:  vio_cnt


In [34]:
print('feature: ', a_2.columns[lasso2.coef_ != 0])

feature:  Index(['house', 'vio_cnt', 'car_cnt', 'popu_u20', 'popu_o60', 'for_u20',
       'for_2030', 'single_u20', 'single_2030', 'single_4050', 'single_o60',
       'single_f_2030', 'for_o40', '교통사고범죄발생대비검거수', '기초질서범죄발생대비검거수',
       '인구수대비검거수', '외국인인구수대비검거수', '인구수대비경찰수', 'before_q1', 'before_q3',
       'before_q4', 'before_q5', 'light_r', 'cctv_r', 'adult_r', 'bell_r',
       'ofn_10', 'ofn_340', 'ofn_m', 'ofn_unknown', 'vic_20', 'vic_80ov',
       'vic_w', 'case_car_car', 'case_car_only'],
      dtype='object')


#### q3 교통안전

In [35]:
# 그리드 서치 후 최고 성능의 모델을 lasso3에 저장

grid_search.fit(xtrain3, ytrain3)
lasso3 = grid_search.best_estimator_

In [36]:
# MAE 출력

y_pred3 = lasso3.predict(xtest3)
mean_absolute_error(ytest3, y_pred3)

2.454345636259885

In [45]:
# 결과

print('alpha =', lasso3.alpha)
print(lasso3.coef_) # Ridge 회귀분석으로 나온 weghit값
print('feature 개수: ', len(a_3.columns[lasso3.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_3.columns[lasso3.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_3.columns[lasso3.coef_.argmin()])

alpha = 0.224
[ 0.54790367  0.          0.          0.         -0.          0.
  0.         -0.          0.          0.          0.          0.
 -0.          0.         -0.         -0.         -0.         -0.
  0.         -0.         -0.          0.          0.         -0.
  0.         -0.          0.          0.          0.          0.
  0.          0.30742698  0.          0.          0.          0.
  0.          0.0555366  -0.         -0.          0.         -0.27066816
 -0.32186105 -0.          0.          0.         -0.          0.
  0.          0.         -0.         -0.          0.         -0.
  0.         -0.         -0.         -0.          0.         -0.        ]
feature 개수:  5
가장 강한 양의 상관관계:  child 
가장 강한 음의 상관관계:  ofn_20


In [38]:
print('feature: ', a_3.columns[lasso3.coef_ != 0])

feature:  Index(['child', '인구수대비경찰수', 'light_r', 'ofn_10', 'ofn_20'], dtype='object')


#### q4 법질서 준수도

In [39]:
# 그리드 서치 후 최고 성능의 모델을 lasso4에 저장

grid_search.fit(xtrain4, ytrain4)
lasso4 = grid_search.best_estimator_

In [40]:
# MAE 출력

y_pred4 = lasso4.predict(xtest4)
mean_absolute_error(ytest4, y_pred4)

2.6412036109058934

In [42]:
# 결과

print('alpha =', lasso4.alpha)
print(lasso4.coef_) # Ridge 회귀분석으로 나온 weghit값
print('feature 개수: ', len(a_4.columns[lasso4.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_4.columns[lasso4.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_4.columns[lasso4.coef_.argmin()])

alpha = 0.097
[ 0.06759591  0.46561362  0.         -0.          0.83671413  0.15551613
 -0.         -0.         -0.         -0.          0.         -0.
 -0.          0.          0.         -1.28720965 -0.          0.
  0.32010344  0.         -0.         -0.         -0.         -0.13990142
  0.         -0.45787246 -0.19417365 -0.          0.00874207 -0.
  0.470805   -1.34447464  0.          0.36514138  0.          1.0184528
  0.          0.          0.08266887 -0.38705872  0.14957544 -0.4219147
 -0.30059636  0.19090082  0.          0.         -0.34446297  0.
  0.35674856  0.20384784 -0.          0.          0.         -0.02388795
  0.         -0.         -0.         -0.08712547  0.28237157 -0.        ]
feature 개수:  26
가장 강한 양의 상관관계:  before_q4 
가장 강한 음의 상관관계:  인구수대비경찰수


In [43]:
print('feature: ', a_4.columns[lasso4.coef_ != 0])

feature:  Index(['child', 'house', 'mur_rob_cnt', 'car_cnt', 'single_4050',
       'single_f_u20', 'for_o40', '절도폭력범죄발생대비검거수', '강도살인범죄발생대비검거수',
       '기초질서범죄발생대비검거수', '외국인인구수대비검거수', '인구수대비경찰수', 'before_q2', 'before_q4',
       'cctv_r', 'adult_r', 'bell_r', 'ofn_10', 'ofn_20', 'ofn_340', 'ofn_m',
       'ofn_unknown', 'vic_10', 'vic_80ov', 'case_car_man', 'case_car_car'],
      dtype='object')


#### q5 전반적 안전도

In [47]:
# 그리드 서치 후 최고 성능의 모델을 lasso4에 저장

grid_search.fit(xtrain5, ytrain5)
lasso5 = grid_search.best_estimator_

In [48]:
# MAE 출력

y_pred5 = lasso5.predict(xtest5)
mean_absolute_error(ytest5, y_pred5)

2.1597428192914614

In [50]:
# 결과

print('alpha =', lasso5.alpha)
print(lasso5.coef_) # Ridge 회귀분석으로 나온 weghit값
print('feature 개수: ', len(a_5.columns[lasso5.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_5.columns[lasso5.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_5.columns[lasso5.coef_.argmin()])

alpha = 0.024
[ 0.          0.45704898  0.         -1.22377051  0.24684585  0.27192982
  0.         -0.          0.59663343  1.07983448  0.68003466 -0.27880184
 -0.          0.21622562  0.         -1.39848521 -0.1524511   0.
  0.          0.63328004  0.         -0.          0.         -0.35744253
  0.         -0.48467528 -0.07423263 -0.          0.0756656  -0.
  0.6920857   0.          0.          0.02679406 -0.11106227  0.09474177
  0.25452695  0.12722076 -0.         -0.18850077  0.06067285 -0.39912854
 -0.24991453  0.23923348 -0.          0.0094063  -0.57496996  0.
  0.23067595 -0.         -0.         -0.          0.05150321 -0.21222363
 -0.          0.06593002 -0.08039202  0.         -0.          0.        ]
feature 개수:  34
가장 강한 양의 상관관계:  popu_o60 
가장 강한 음의 상관관계:  single_4050


In [51]:
print('feature: ', a_5.columns[lasso5.coef_ != 0])

feature:  Index(['house', 'vio_cnt', 'mur_rob_cnt', 'car_cnt', 'popu_u20', 'popu_o60',
       'for_u20', 'for_2030', 'single_u20', 'single_4050', 'single_o60',
       'single_f_2030', 'for_o40', '절도폭력범죄발생대비검거수', '강도살인범죄발생대비검거수',
       '기초질서범죄발생대비검거수', '외국인인구수대비검거수', 'before_q2', 'before_q3', 'before_q4',
       'before_q5', 'light_r', 'adult_r', 'bell_r', 'ofn_10', 'ofn_20',
       'ofn_340', 'ofn_80ov', 'ofn_m', 'ofn_unknown', 'vic_5670', 'vic_80ov',
       'vic_w', 'vic_unknown'],
      dtype='object')
